## Forecasting when faced with thousands of time series

* Simulate various time series
* Cluster them 
* Assign models to the clusters
* Plot performance metric distribution
* For example time series, compare cluster-derived model with indvidually fit model

The idea behind most models is to reduce the input data to random noise. What I mean by that is that you want to explain as much of the variation in the input data as you possibly can while using the model. So if you were to subtract the parts of the data which were successfully modeled from the input data, you would want to end up with a portion that has no structure left to it, and resembles independent and identically distributed random noise. This is the idea behind 

In [1]:
import tslearn
import hdbscan 

# has warping path visualization functionality
import dtaidistance # only calculates euclidean distance to speed up C-code underneath by avoiding extra function calls
from dtaidistance import dtw_visualisation as dtwvis

import fastdtw
import statsmodels.api as sm
from statsmodels.tsa.arima_process import arma_generate_sample
import numpy as np
import pandas as pd
import scipy as sp
import random
import matplotlib.pyplot as plt
%matplotlib inline

from time_series_simulations import time_series_simulations

In [2]:
standardized_series = time_series_simulations.simulate_ts_population()

## Calculate pairwise distances

In [3]:
# dtaidistance has a built in function that produces a pairwise distance matrix 
# first put all the series in a list of lists 
series_list = standardized_series.groupby('id')['stand_amount'].apply(np.array) # returns a series
series_list = [np.array(x) for x in series_list] # cast series into a list of np.arrays
print(series_list[0])

[-0.72685071  0.52563225  0.95930649 -1.56256248  0.45150355 -0.71395721
  0.40762941  0.88821814  0.92704023  0.02040741 -0.16955663  0.5874664
 -0.61815856  0.56904348 -1.45915175  0.16842142  1.51584452  0.36570559
  0.34337022  0.07911277  1.63906113  0.8220618   1.28789809 -0.62880688
 -1.20176961  0.48687677 -0.92090982  0.42800492  0.1558313   0.55878002
 -1.18562172 -0.57440987 -0.79067984 -1.7945497  -0.21673961  0.53757386
 -1.42324141 -1.32781774 -0.59417461 -1.70969325 -0.28448457  0.22150796
  0.71791662 -1.55781998  0.30037301  0.25301397  2.31896862  1.61849892
 -0.3563183   0.2569566  -0.69207831 -0.66195407  1.40825923  0.82585874
  0.31476634  1.51400186 -0.35623994 -0.32276455  1.15329419  0.51057827
  2.52435865  1.39170539  1.25300208 -0.27805596 -0.4274982   0.04867993
  0.04055038 -1.71015166 -1.23078383  0.27622799 -0.04998011  0.71404242
  0.34902176 -0.70697789  1.98689171  1.65046206 -2.4003277  -2.44305963
  0.51793546 -0.09266708 -1.32180144 -0.18299565 -1.

In [4]:
# then calculate the matrix
distance_matrix = dtaidistance.dtw.distance_matrix_fast(series_list)

In [5]:
# alternatively, here is my code to do the pairwise calculations without a library
# this is probably slower than dtaidistance.dtw.distance_matrix_fast 
# create an empty square dataframe to populate with distances
n_accounts = len(np.unique(standardized_series['id'])) 
pairwise_dtw = pd.DataFrame(np.zeros([n_accounts, n_accounts])) 

In [6]:
n_accounts

1400

In [ ]:
# get the iterable
the_ids = standardized_series['id']

# loop through the iterable
for i, id1 in enumerate(the_ids):
    for j, id2 in enumerate(the_ids): # only populate top half 
        # get the accounts' time series
        ts_one = standardized_series.loc[standardized_series['id'] == id1, ['timepoint', 'amount']]
        ts_two = standardized_series.loc[standardized_series['id'] == id2, ['timepoint', 'amount']]
        
        # calculate Manhattan distance
        pairwise_dtw.loc[i,j] = fastdtw.fastdtw(ts_one, ts_two, dist=1)[0]
        print(i,id1, "\t",j, id2)
        
        # calculate Euclidean distance
        #pairwise_dtw.loc[i,j] = dtaidistance.dtw.distance(ts_one['amount'], ts_two['amount'])

/Users/athenahadjixenofontos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if sys.path[0] == '':


0 0ar1 	 0 0ar1
0 0ar1 	 1 0ar1
0 0ar1 	 2 0ar1
0 0ar1 	 3 0ar1
0 0ar1 	 4 0ar1
0 0ar1 	 5 0ar1
0 0ar1 	 6 0ar1
0 0ar1 	 7 0ar1
0 0ar1 	 8 0ar1
0 0ar1 	 9 0ar1
0 0ar1 	 10 0ar1
0 0ar1 	 11 0ar1
0 0ar1 	 12 0ar1
0 0ar1 	 13 0ar1
0 0ar1 	 14 0ar1
0 0ar1 	 15 0ar1
0 0ar1 	 16 0ar1
0 0ar1 	 17 0ar1
0 0ar1 	 18 0ar1
0 0ar1 	 19 0ar1
0 0ar1 	 20 0ar1
0 0ar1 	 21 0ar1
0 0ar1 	 22 0ar1
0 0ar1 	 23 0ar1
0 0ar1 	 24 0ar1
0 0ar1 	 25 0ar1
0 0ar1 	 26 0ar1
0 0ar1 	 27 0ar1
0 0ar1 	 28 0ar1
0 0ar1 	 29 0ar1
0 0ar1 	 30 0ar1
0 0ar1 	 31 0ar1
0 0ar1 	 32 0ar1
0 0ar1 	 33 0ar1
0 0ar1 	 34 0ar1
0 0ar1 	 35 0ar1
0 0ar1 	 36 0ar1
0 0ar1 	 37 0ar1
0 0ar1 	 38 0ar1
0 0ar1 	 39 0ar1
0 0ar1 	 40 0ar1
0 0ar1 	 41 0ar1
0 0ar1 	 42 0ar1
0 0ar1 	 43 0ar1
0 0ar1 	 44 0ar1
0 0ar1 	 45 0ar1
0 0ar1 	 46 0ar1
0 0ar1 	 47 0ar1
0 0ar1 	 48 0ar1
0 0ar1 	 49 0ar1
0 0ar1 	 50 0ar1
0 0ar1 	 51 0ar1
0 0ar1 	 52 0ar1
0 0ar1 	 53 0ar1
0 0ar1 	 54 0ar1
0 0ar1 	 55 0ar1
0 0ar1 	 56 0ar1
0 0ar1 	 57 0ar1
0 0ar1 	 58 0ar1
0 0ar1 

0 0ar1 	 396 0arima111
0 0ar1 	 397 0arima111
0 0ar1 	 398 0arima111
0 0ar1 	 399 0arima111
0 0ar1 	 400 0arima111
0 0ar1 	 401 0arima111
0 0ar1 	 402 0arima111
0 0ar1 	 403 0arima111
0 0ar1 	 404 0arima111
0 0ar1 	 405 0arima111
0 0ar1 	 406 0arima111
0 0ar1 	 407 0arima111
0 0ar1 	 408 0arima111
0 0ar1 	 409 0arima111
0 0ar1 	 410 0arima111
0 0ar1 	 411 0arima111
0 0ar1 	 412 0arima111
0 0ar1 	 413 0arima111
0 0ar1 	 414 0arima111
0 0ar1 	 415 0arima111
0 0ar1 	 416 0arima111
0 0ar1 	 417 0arima111
0 0ar1 	 418 0arima111
0 0ar1 	 419 0arima111
0 0ar1 	 420 0arima111
0 0ar1 	 421 0arima111
0 0ar1 	 422 0arima111
0 0ar1 	 423 0arima111
0 0ar1 	 424 0arima111
0 0ar1 	 425 0arima111
0 0ar1 	 426 0arima111
0 0ar1 	 427 0arima111
0 0ar1 	 428 0arima111
0 0ar1 	 429 0arima111
0 0ar1 	 430 0arima111
0 0ar1 	 431 0arima111
0 0ar1 	 432 0arima111
0 0ar1 	 433 0arima111
0 0ar1 	 434 0arima111
0 0ar1 	 435 0arima111
0 0ar1 	 436 0arima111
0 0ar1 	 437 0arima111
0 0ar1 	 438 0arima111
0 0ar1 	 43

0 0ar1 	 760 0arima211
0 0ar1 	 761 0arima211
0 0ar1 	 762 0arima211
0 0ar1 	 763 0arima211
0 0ar1 	 764 0arima211
0 0ar1 	 765 0arima211
0 0ar1 	 766 0arima211
0 0ar1 	 767 0arima211
0 0ar1 	 768 0arima211
0 0ar1 	 769 0arima211
0 0ar1 	 770 0arima211
0 0ar1 	 771 0arima211
0 0ar1 	 772 0arima211
0 0ar1 	 773 0arima211
0 0ar1 	 774 0arima211
0 0ar1 	 775 0arima211
0 0ar1 	 776 0arima211
0 0ar1 	 777 0arima211
0 0ar1 	 778 0arima211
0 0ar1 	 779 0arima211
0 0ar1 	 780 0arima211
0 0ar1 	 781 0arima211
0 0ar1 	 782 0arima211
0 0ar1 	 783 0arima211
0 0ar1 	 784 0arima222
0 0ar1 	 785 0arima222
0 0ar1 	 786 0arima222
0 0ar1 	 787 0arima222
0 0ar1 	 788 0arima222
0 0ar1 	 789 0arima222
0 0ar1 	 790 0arima222
0 0ar1 	 791 0arima222
0 0ar1 	 792 0arima222
0 0ar1 	 793 0arima222
0 0ar1 	 794 0arima222
0 0ar1 	 795 0arima222
0 0ar1 	 796 0arima222
0 0ar1 	 797 0arima222
0 0ar1 	 798 0arima222
0 0ar1 	 799 0arima222
0 0ar1 	 800 0arima222
0 0ar1 	 801 0arima222
0 0ar1 	 802 0arima222
0 0ar1 	 80

0 0ar1 	 1132 0arma12
0 0ar1 	 1133 0arma12
0 0ar1 	 1134 0arma12
0 0ar1 	 1135 0arma12
0 0ar1 	 1136 0arma12
0 0ar1 	 1137 0arma12
0 0ar1 	 1138 0arma12
0 0ar1 	 1139 0arma12
0 0ar1 	 1140 0arma12
0 0ar1 	 1141 0arma12
0 0ar1 	 1142 0arma12
0 0ar1 	 1143 0arma12
0 0ar1 	 1144 0arma12
0 0ar1 	 1145 0arma12
0 0ar1 	 1146 0arma12
0 0ar1 	 1147 0arma12
0 0ar1 	 1148 0arma12
0 0ar1 	 1149 0arma12
0 0ar1 	 1150 0arma12
0 0ar1 	 1151 0arma12
0 0ar1 	 1152 0arma12
0 0ar1 	 1153 0arma12
0 0ar1 	 1154 0arma12
0 0ar1 	 1155 0arma12
0 0ar1 	 1156 0arma12
0 0ar1 	 1157 0arma12
0 0ar1 	 1158 0arma12
0 0ar1 	 1159 0arma12
0 0ar1 	 1160 0arma12
0 0ar1 	 1161 0arma12
0 0ar1 	 1162 0arma12
0 0ar1 	 1163 0arma12
0 0ar1 	 1164 0arma12
0 0ar1 	 1165 0arma12
0 0ar1 	 1166 0arma12
0 0ar1 	 1167 0arma12
0 0ar1 	 1168 0arma12
0 0ar1 	 1169 0arma12
0 0ar1 	 1170 0arma12
0 0ar1 	 1171 0arma12
0 0ar1 	 1172 0arma12
0 0ar1 	 1173 0arma12
0 0ar1 	 1174 0arma12
0 0ar1 	 1175 0arma12
0 0ar1 	 1176 0arma12
0 0ar1 	 1

...

0 0ar1 	 17694 20arima112
0 0ar1 	 17695 20arima112
0 0ar1 	 17696 20arima112
0 0ar1 	 17697 20arima112
0 0ar1 	 17698 20arima112
0 0ar1 	 17699 20arima112
0 0ar1 	 17700 20arima112
0 0ar1 	 17701 20arima112
0 0ar1 	 17702 20arima112
0 0ar1 	 17703 20arima112
0 0ar1 	 17704 20arima112
0 0ar1 	 17705 20arima112
0 0ar1 	 17706 20arima112
0 0ar1 	 17707 20arima112
0 0ar1 	 17708 20arima112
0 0ar1 	 17709 20arima112
0 0ar1 	 17710 20arima112
0 0ar1 	 17711 20arima112
0 0ar1 	 17712 20arima112
0 0ar1 	 17713 20arima112
0 0ar1 	 17714 20arima112
0 0ar1 	 17715 20arima112
0 0ar1 	 17716 20arima112
0 0ar1 	 17717 20arima112
0 0ar1 	 17718 20arima112
0 0ar1 	 17719 20arima112
0 0ar1 	 17720 20arima112
0 0ar1 	 17721 20arima112
0 0ar1 	 17722 20arima112
0 0ar1 	 17723 20arima112
0 0ar1 	 17724 20arima112
0 0ar1 	 17725 20arima112
0 0ar1 	 17726 20arima112
0 0ar1 	 17727 20arima112
0 0ar1 	 17728 20arima112
0 0ar1 	 17729 20arima112
0 0ar1 	 17730 20arima112
0 0ar1 	 17731 20arima112
0 0ar1 	 177

0 0ar1 	 18020 20arma12
0 0ar1 	 18021 20arma12
0 0ar1 	 18022 20arma12
0 0ar1 	 18023 20arma12
0 0ar1 	 18024 20arma12
0 0ar1 	 18025 20arma12
0 0ar1 	 18026 20arma12
0 0ar1 	 18027 20arma12
0 0ar1 	 18028 20arma12
0 0ar1 	 18029 20arma12
0 0ar1 	 18030 20arma12
0 0ar1 	 18031 20arma12
0 0ar1 	 18032 20arma12
0 0ar1 	 18033 20arma12
0 0ar1 	 18034 20arma12
0 0ar1 	 18035 20arma12
0 0ar1 	 18036 20arma12
0 0ar1 	 18037 20arma12
0 0ar1 	 18038 20arma12
0 0ar1 	 18039 20arma12
0 0ar1 	 18040 20arma12
0 0ar1 	 18041 20arma12
0 0ar1 	 18042 20arma12
0 0ar1 	 18043 20arma12
0 0ar1 	 18044 20arma12
0 0ar1 	 18045 20arma21
0 0ar1 	 18046 20arma21
0 0ar1 	 18047 20arma21
0 0ar1 	 18048 20arma21
0 0ar1 	 18049 20arma21
0 0ar1 	 18050 20arma21
0 0ar1 	 18051 20arma21
0 0ar1 	 18052 20arma21
0 0ar1 	 18053 20arma21
0 0ar1 	 18054 20arma21
0 0ar1 	 18055 20arma21
0 0ar1 	 18056 20arma21
0 0ar1 	 18057 20arma21
0 0ar1 	 18058 20arma21
0 0ar1 	 18059 20arma21
0 0ar1 	 18060 20arma21
0 0ar1 	 18061 2

0 0ar1 	 18394 21ar2
0 0ar1 	 18395 21ar2
0 0ar1 	 18396 21ar2
0 0ar1 	 18397 21ar2
0 0ar1 	 18398 21ar2
0 0ar1 	 18399 21ar2
0 0ar1 	 18400 21ar2
0 0ar1 	 18401 21ar2
0 0ar1 	 18402 21ar2
0 0ar1 	 18403 21ar2
0 0ar1 	 18404 21ar2
0 0ar1 	 18405 21ar2
0 0ar1 	 18406 21ar2
0 0ar1 	 18407 21ar2
0 0ar1 	 18408 21ar2
0 0ar1 	 18409 21ar2
0 0ar1 	 18410 21ar2
0 0ar1 	 18411 21ar2
0 0ar1 	 18412 21ar2
0 0ar1 	 18413 21ar2
0 0ar1 	 18414 21ar2
0 0ar1 	 18415 21ar2
0 0ar1 	 18416 21ar2
0 0ar1 	 18417 21ar2
0 0ar1 	 18418 21ar2
0 0ar1 	 18419 21ar2
0 0ar1 	 18420 21ar2
0 0ar1 	 18421 21ar2
0 0ar1 	 18422 21ar2
0 0ar1 	 18423 21ar2
0 0ar1 	 18424 21ar2
0 0ar1 	 18425 21ar2
0 0ar1 	 18426 21ar2
0 0ar1 	 18427 21ar2
0 0ar1 	 18428 21ar2
0 0ar1 	 18429 21ar2
0 0ar1 	 18430 21ar2
0 0ar1 	 18431 21ar2
0 0ar1 	 18432 21ar2
0 0ar1 	 18433 21ar2
0 0ar1 	 18434 21ar2
0 0ar1 	 18435 21ar2
0 0ar1 	 18436 21ar2
0 0ar1 	 18437 21ar2
0 0ar1 	 18438 21ar2
0 0ar1 	 18439 21ar2
0 0ar1 	 18440 21ar2
0 0ar1 	 1844

0 0ar1 	 18726 21arima110
0 0ar1 	 18727 21arima110
0 0ar1 	 18728 21arima110
0 0ar1 	 18729 21arima110
0 0ar1 	 18730 21arima110
0 0ar1 	 18731 21arima110
0 0ar1 	 18732 21arima110
0 0ar1 	 18733 21arima110
0 0ar1 	 18734 21arima110
0 0ar1 	 18735 21arima110
0 0ar1 	 18736 21arima110
0 0ar1 	 18737 21arima110
0 0ar1 	 18738 21arima110
0 0ar1 	 18739 21arima110
0 0ar1 	 18740 21arima110
0 0ar1 	 18741 21arima110
0 0ar1 	 18742 21arima111
0 0ar1 	 18743 21arima111
0 0ar1 	 18744 21arima111
0 0ar1 	 18745 21arima111
0 0ar1 	 18746 21arima111
0 0ar1 	 18747 21arima111
0 0ar1 	 18748 21arima111
0 0ar1 	 18749 21arima111
0 0ar1 	 18750 21arima111
0 0ar1 	 18751 21arima111
0 0ar1 	 18752 21arima111
0 0ar1 	 18753 21arima111
0 0ar1 	 18754 21arima111
0 0ar1 	 18755 21arima111
0 0ar1 	 18756 21arima111
0 0ar1 	 18757 21arima111
0 0ar1 	 18758 21arima111
0 0ar1 	 18759 21arima111
0 0ar1 	 18760 21arima111
0 0ar1 	 18761 21arima111
0 0ar1 	 18762 21arima111
0 0ar1 	 18763 21arima111
0 0ar1 	 187

0 0ar1 	 19042 21arima112
0 0ar1 	 19043 21arima112
0 0ar1 	 19044 21arima112
0 0ar1 	 19045 21arima112
0 0ar1 	 19046 21arima112
0 0ar1 	 19047 21arima112
0 0ar1 	 19048 21arima112
0 0ar1 	 19049 21arima112
0 0ar1 	 19050 21arima112
0 0ar1 	 19051 21arima112
0 0ar1 	 19052 21arima112
0 0ar1 	 19053 21arima112
0 0ar1 	 19054 21arima112
0 0ar1 	 19055 21arima112
0 0ar1 	 19056 21arima112
0 0ar1 	 19057 21arima112
0 0ar1 	 19058 21arima112
0 0ar1 	 19059 21arima112
0 0ar1 	 19060 21arima112
0 0ar1 	 19061 21arima112
0 0ar1 	 19062 21arima112
0 0ar1 	 19063 21arima112
0 0ar1 	 19064 21arima112
0 0ar1 	 19065 21arima112
0 0ar1 	 19066 21arima112
0 0ar1 	 19067 21arima112
0 0ar1 	 19068 21arima112
0 0ar1 	 19069 21arima112
0 0ar1 	 19070 21arima112
0 0ar1 	 19071 21arima112
0 0ar1 	 19072 21arima112
0 0ar1 	 19073 21arima112
0 0ar1 	 19074 21arima112
0 0ar1 	 19075 21arima112
0 0ar1 	 19076 21arima112
0 0ar1 	 19077 21arima112
0 0ar1 	 19078 21arima112
0 0ar1 	 19079 21arima112
0 0ar1 	 190

#### Set up the simulated series in the wide format for clustering

In [61]:
# TODO fix this if necessary, use the series_list above as a starting point
## clustering wants the data in the wide format
# n_rows = len([element for sim_list in all_series for element in sim_list])

# all_series_wide = pd.DataFrame(index = range(0, n_rows), columns = ['id', 'timepoint', 'amount', 'process'])

# for index in range(0, len(all_series)):
#     all_series_wide.at[index, 'id'] = index
#     all_series_wide.at[index, 'timepoint'] = list(range(0, len(all_series[index])))
#     all_series_wide.at[index, 'amount'] = list(all_series[index])
#     all_series_wide.at[index, 'process'] = 

In [60]:
# sim_series_wide.head()

As an alternative to warped paths, I could align in time and then calculate Euclidean distance timepoint by timepoint. This would be more appropriate if the series have seasonality - perhaps. It's also possible that if the series do have seasonality, they are 

### Cluster

Two options: 

1. Use a supervised method by training a 1-NN classifier that uses DTW distances, 5-fold cross validation (based on sample size of 100) and simulated data so that the observations (time series) have labels (the process from which they were simulated eg. ARMA(1,1)). Hold out some of the simulated data to use as the test set, this represents the unlabeled "real" data. 
    
    1-nn and dtw is difficult to beat for time series classification, and for a good summary of approaches taken towards time series classification including neural networks (predictive modular NNs, and a combination of HMM and ANNs) (page 83) [Mitsa et al 2010 Temporal Data Mining chapter on Temporal Classification](https://books.google.com/books?id=4P_7ydvW7cAC&pg=PA67&source=gbs_toc_r&cad=3#v=onepage&q&f=false)

    A time series can be represented by a feature vector made up of the Fourier transform coefficients of the time series, and its mean, skewness, and kurtosis. This feature vector can be used as input to a clustering algorithm. 

    Also see [this paper](https://arxiv.org/pdf/1406.4757.pdf)

2. Alternatively, use hierarchical clustering to identify groups of similar time series in an unsupervised way. The hope would be for time series generated by the same process, e.g. an ARMA(1,1) to be more similar to each other than to those generated by other processes, e.g. a MA(2). 

### With dtaidistance clustering 

In [9]:
from dtaidistance import clustering
from numpy import inf

In [10]:
distance_matrix[distance_matrix == inf] = 0

In [11]:
distance_matrix

array([[ 0.        ,  7.56692732,  8.18123984, ...,  9.17965595,
        10.16301923,  7.97051795],
       [ 0.        ,  0.        , 12.19850756, ..., 16.60986772,
         8.46423706,  7.26162304],
       [ 0.        ,  0.        ,  0.        , ..., 12.15632332,
        13.29063069, 11.41954925],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        16.85887797, 16.10557976],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  7.95665619],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
# # computes the distance matrix first 
# model = dtaidistance.clustering.Hierarchical(dtaidistance.dtw.distance_matrix_fast, {})
# model2 = dtaidistance.clustering.HierarchicalTree(model)
# cluster_idx = model2.fit(distance_matrix)

### With sklearn and scipy

In [65]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import DistanceMetric

from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

In [70]:
# with sklearn
n_clusters = len(np.unique(list(subset_series['id']))) # use the real number of clusters 
hclust = AgglomerativeClustering(n_clusters = n_clusters, affinity='precomputed', linkage = 'average')
clusters = hclust.fit(distance_matrix)

array([0, 0, 0, ..., 0, 0, 0])

In [77]:
# get a frequency table of the labels 
# np.bincount(clusters.labels_)
np.unique(clusters.labels_, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]),
 array([1381,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1]))

These results are not very good. A single observation per cluster when n_cluster ranges from 4 (reasonable based on the ARMA processes simulated) to 20 indicates that the time series are too dissimilar, which is in line with the dtw paths we saw above. 

In [ ]:
# with scipy
scipy_hclust = linkage(pairwise_dtw, 'average')
fig = plt.figure(figsize=(25, 10))
dendro = dendrogram(hclust)

In [ ]:
scipy_hclust # what do these represent?

In [ ]:
# get cluster labels
results = dendrogram(scipy_hclust, 
                    truncate_mode = 'lastp',  # show only the last p merged clusters
                    p = 4, # show only the last 4 merged clusters 
                    no_plot = True) # returns the dictionary that contains the labels instead of the plot

In [ ]:
results

In [ ]:
fcluster(hclust, criterion='distance') # get cluster labels

In [ ]:
# try to decompose the series and cluster only the trend components in an attempt to reduce the noise?
# or go with the supervised classification approach 
# or ARMA processes are too similar to each other to tease apart the p and q parameters based on the geometry of the series - add ARCH/GARCH processes to the mix to see

In [ ]:
# # use custom distance metric
# dtw_metric = DistanceMetric.get_metric('pyfunc', func = fastdtw(dist = 1))
# # 1-NN
# knn = NearestNeighbors(n_neighbors = 1, radius = 0.5)#, metric = dtw_metric)
# knn.fit(sim_series)
